# Preprocess & Embed with tGPT

In [1]:
from embed import *

In [2]:
# load our data
input_file = '/nfs/turbo/umms-indikar/shared/projects/geneformer/data/rajapakse_lab_data_jpic.h5ad'
adata         = sc.read_h5ad(input_file)

In [3]:
embedAd = embed(adata, 'tGPT', output_directory='~', output_file='tgpt.h5ad')

cuda
Tokenizer set


/home/jpic/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


model set
model to device
model eval


100%|██████████| 2/2 [00:00<00:00,  2.89it/s]


In [4]:
embedAd.obs

,dataset,sample_id,timepoint,hour,n_counts,control,order,replicate,batch
data_id,,,,,,,,,
S1a,chen_2015,S1a,0.0,0.0,7901832,True,1,1,0
S1b,chen_2015,S1b,0.0,0.0,8113329,True,1,2,0
S2a,chen_2015,S2a,0.0,0.0,9831046,False,2,1,0
S2b,chen_2015,S2b,0.0,0.0,10123271,False,2,2,0
S3a,chen_2015,S3a,1.0,8.0,10490839,False,3,1,0
...,...,...,...,...,...,...,...,...,...
63275_T11R3,liu_2018,63275,3.0,80.0,13515971,False,11,3,1
63290_T12R3,liu_2018,63290,1.0,88.0,9522866,False,12,3,1
63287_T13R3,liu_2018,63287,2.0,96.0,12370157,False,13,3,1


In [5]:
embedAdGF = embed(adata, 'geneformer', output_directory='~', output_file='tgpt.h5ad')

Map (num_proc=16):   0%|          | 0/66 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/66 [00:00<?, ? examples/s]

/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/


  0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
embedAdGF

AnnData object with n_obs × n_vars = 66 × 512
    obs: 'dataset', 'sample_id', 'timepoint', 'hour', 'n_counts', 'control', 'order', 'replicate', 'batch'

In [7]:
embedAd

AnnData object with n_obs × n_vars = 66 × 1024
    obs: 'dataset', 'sample_id', 'timepoint', 'hour', 'n_counts', 'control', 'order', 'replicate', 'batch'

In [ ]:
from gfUtils import *

In [1]:
# cuda appears particularlly finicky with this model. If cuda.synchrnize() doesn't run, then model won't run
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))
print(torch.cuda.synchronize())

#device = torch.device('cuda:0')
#print(f'Allocated Memory: {get_human_readable_size(torch.cuda.memory_allocated(device))}')
#print(f'Reserved Memory: {get_human_readable_size(torch.cuda.memory_reserved(device))}')



True
1
0
Tesla V100-PCIE-16GB
None


In [2]:
import torch

In [5]:
import re
import os
import sys
import gzip
import torch
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as an
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2Model

In [4]:
# load our data
input_file = '/nfs/turbo/umms-indikar/shared/projects/geneformer/data/rajapakse_lab_data_jpic.h5ad'
adata         = sc.read_h5ad(input_file)

In [9]:
class LineDataset(Dataset):
    def __init__(self, lines):
        self.lines = lines
        self.regex = re.compile(r'\-|\.')
    def __getitem__(self, i):
        return self.regex.sub('_', self.lines[i])
    def __len__(self):
        return len(self.lines)

def get_top_genes_per_cell(adata, top_n=256):
    # Extract the data matrix
    X = adata.X
    
    # Initialize a list to hold the top genes for each cell
    top_genes_per_cell = []

    for i in range(X.shape[0]):
        # Get the expression values for the current cell
        cell_expression = X[i, :]
        
        # Get the indices of the top N expression values
        top_gene_indices = np.argsort(cell_expression)[-top_n:]
        
        # Get the gene names corresponding to the top indices
        top_gene_names = adata.var_names[top_gene_indices]
        
        top_genes_per_cell.append(" ".join(list(top_gene_names)))
    
    return top_genes_per_cell

def tGPTembed(adata):

    topGenes = get_top_genes_per_cell(adata)

    ds = LineDataset(topGenes)
    dl = DataLoader(ds, batch_size=64)
    
    Xs = []

    # load some tGPT params
    device = "cuda" if torch.cuda.is_available() else "cpu" 
    print(device)
    tokenizer_file = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/transcriptome-gpt-1024-8-16-64" 
    checkpoint     = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/transcriptome-gpt-1024-8-16-64" ## Pretrained model
    celltype_path  = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/example_inputs/tGPT/Muris_cell_labels.txt.gz" ## Cell type annotation
    max_len        = 64 ## Number of top genes used for analysis
    text_file      = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/example_inputs/tGPT/Muris_gene_rankings.txt.gz"  ## Gene symbols ranked by exprssion
    
    tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_file)
    print('Tokenizer set')
    model = GPT2LMHeadModel.from_pretrained(checkpoint,output_hidden_states = True).transformer
    print('model set')
    model = model.to(device)
    print('model to device')
    model.eval()
    print('model eval')

    for a in tqdm(dl, total=len(dl)):
        batch = tokenizer(a, max_length= max_len, truncation=True, padding=True, return_tensors="pt")
    
        for k, v in batch.items():
            batch[k] = v.to(device)
    
        with torch.no_grad():
            x = model(**batch)
        
        eos_idxs = batch.attention_mask.sum(dim=1) - 1
        xx = x.last_hidden_state
           
        result_list = [[] for i in range(len(xx))]
    
        for j, item in enumerate(xx):
            result_list[j] = item[1:int(eos_idxs[j]),:].mean(dim =0).tolist()
            
        Xs.extend(result_list)
        
    features = np.stack(Xs)

    emebddingAdata = ad.AnnData(X=features)

xx = tGPTembed(adata)

cuda
Tokenizer set
model set
model to device
model eval


100%|██████████| 2/2 [00:00<00:00, 17.82it/s]


In [4]:
# load some tGPT params
device = "cuda" if torch.cuda.is_available() else "cpu" 
print(device)
tokenizer_file = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/transcriptome-gpt-1024-8-16-64" 
checkpoint     = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/transcriptome-gpt-1024-8-16-64" ## Pretrained model
celltype_path  = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/example_inputs/tGPT/Muris_cell_labels.txt.gz" ## Cell type annotation
max_len        = 64 ## Number of top genes used for analysis
text_file      = "/nfs/turbo/umms-indikar/shared/projects/foundation_models/example_inputs/tGPT/Muris_gene_rankings.txt.gz"  ## Gene symbols ranked by exprssion

cuda


In [25]:
def get_top_genes_per_cell(adata, top_n=256):
    # Extract the data matrix
    X = adata.X
    
    # Initialize a list to hold the top genes for each cell
    top_genes_per_cell = []

    for i in range(X.shape[0]):
        # Get the expression values for the current cell
        cell_expression = X[i, :]
        
        # Get the indices of the top N expression values
        top_gene_indices = np.argsort(cell_expression)[-top_n:]
        
        # Get the gene names corresponding to the top indices
        top_gene_names = adata.var_names[top_gene_indices]
        
        top_genes_per_cell.append(" ".join(list(top_gene_names)))
    
    return top_genes_per_cell

In [26]:
topGenes = get_top_genes_per_cell(adata)

ds = LineDataset(topGenes)
dl = DataLoader(ds, batch_size=64)

Xs = []

In [24]:
len(lines[3].split(' '))

256

In [5]:
class LineDataset(Dataset):
    def __init__(self, lines):
        self.lines = lines
        self.regex = re.compile(r'\-|\.')
    def __getitem__(self, i):
        return self.regex.sub('_', self.lines[i])
    def __len__(self):
        return len(self.lines)

tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_file)
print('Tokenizer set')
model = GPT2LMHeadModel.from_pretrained(checkpoint,output_hidden_states = True).transformer
print('model set')
model = model.to(device)
print('model to device')
model.eval()
print('model eval')

lines = [s.decode().strip() for s in gzip.open(text_file, "r").readlines()]

ds = LineDataset(lines)
dl = DataLoader(ds, batch_size=64)

Xs = []

Tokenizer set


/home/jpic/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


model set
model to device
model eval


In [27]:
for a in tqdm(dl, total=len(dl)):
    batch = tokenizer(a, max_length= max_len, truncation=True, padding=True, return_tensors="pt")

    for k, v in batch.items():
        batch[k] = v.to(device)

    with torch.no_grad():
        x = model(**batch)
    
    eos_idxs = batch.attention_mask.sum(dim=1) - 1
    xx = x.last_hidden_state
       
    result_list = [[] for i in range(len(xx))]

    for j, item in enumerate(xx):
        result_list[j] = item[1:int(eos_idxs[j]),:].mean(dim =0).tolist()
        
    Xs.extend(result_list)
    
features = np.stack(Xs)

100%|██████████| 2/2 [00:06<00:00,  3.22s/it]


In [29]:
features.shape

(66, 1024)

In [4]:
adata=sc.AnnData(features)
celltype = pd.read_csv(celltype_path, header=None)[0].tolist()
adata.obs["celltype"] = celltype
adata.obs["celltype"] = adata.obs["celltype"].astype("category")
adata

AnnData object with n_obs × n_vars = 54865 × 1024
    obs: 'celltype'

In [6]:
adata.obs

,celltype
0,myeloid cell
1,alveolar macrophage
2,B cell
3,natural killer cell
4,T cell
...,...
54860,endothelial cell
54861,endothelial cell
54862,mesenchymal stem cell
54863,mesenchymal stem cell


In [7]:
lines

['CD74 ACTB CST3 TMSB4X EEF1A1 TMSB10 UBA52 FAU TPT1 PSAP B2M GM2A FTH1 ACTG1 HSPA8 PFN1 CRIP1 LY6E H3F3B TYROBP CD52 PPT1 ID2 ZFP36 EEF1B2 PPIA PTMA EEF2 EIF1 DUSP1 SERF2 UCP2 LSP1 GPX1 SH3BGRL3 CFL1 IFITM3 ATOX1 GNG10 HSP90AB1 ACTR3 IRF8 NACA LRRC58 YBX1 BTG1 JUND S100A11 ARPC1B PSME1 UBB ITM2B PLAC8 LAPTM5 COX7C CDC42 PTMS NAAA PABPC1 PSMB8 MYL6 PLBD1 ARPC2 H3F3A GDI2 CORO1A DDX5 BTF3 AKR1A1 CYBA JUNB CALM1 GNB2 MGP GNG5 CD83 BTG2 TALDO1 SRGN NPC2 FOS NOTCH2 COX7A2L EIF5A CYCS TSC22D3 UQCRH EIF3H GNAI2 TOMM7 PRDX1 ATP5G2 CHCHD2 COX6C WDR89 HNRNPA3 ATPIF1 ATP5E COX6B1 EFHD2 ATP5J CKB COX4I1 PKIB AIF1 EIF3F CLEC9A PFDN5 HNRNPF CALR PSMA7 OAZ1 FAM105A POMP LDHA WDFY4 SERP1 SNRPD2 FGD2 BAX DBI RGS2 MPEG1 ALOX5AP FXYD5 USE1 SEC61G ATP5B CYTIP H2AFZ ITGB1 CLIC1 ATP5C1 NDUFA4 SRSF5 NDUFA3 SEC61B NEDD8 ARPC3 RAC2 SFTPC HNRNPA2B1 NFKBIA UBC SUB1 ARPC4 GNB1 ARF6 TGFBI PARK7 SHISA5 SELPLG HEXA MYL12B HERPUD1 TXNIP PSMB9 UQCR11 TAPBP CTSS IFI30 PPP1CA EHBP1L1 CCDC12 TPM4 IFITM2 CD48 COX8A GABAR